In [3]:
import requests
import json
import time
import pandas as pd
import urllib3
import plotly.graph_objects as go
import numpy as np
import plotly
plotly.offline.init_notebook_mode(connected=True)
%matplotlib inline

In [16]:
def convertTime(timestamp_list):
    time_current_list = []
    for time_stamp in timestamp_list:
        time_local = time.localtime(time_stamp)
        time_current = time.strftime("%Y-%m-%d %H:%M:%S",time_local)
        time_current_list.append(time_current)
    return time_current_list
def convertFahrenheit(fahrenheit_list):
    celsius_list = []
    for fahrenheit in fahrenheit_list:
        celsius = round((fahrenheit - 32) / 1.8,1)
        celsius_list.append(celsius)
    return celsius_list

def getApiData(url):
    payload = {}
    files = {}
    headers= {'Accept-Encoding': 'gzip'}
    session = requests.Session()
    response = session.request("GET", url, headers=headers, data = payload, files = files)
    responseJson = json.loads(response.text)
    return responseJson

In [30]:
indicator_list = ['SP.POP.TOTL','SL.AGR.EMPL.ZS','NY.GDP.MKTP.CD',
            'DPANUSSPF','EG.ELC.ACCS.UR.ZS','EG.ELC.ACCS.RU.ZS']
countryAll = ';'.join(['BR','RU','IN','CN','ZA'])

In [107]:
indicator_list = ['SP.POP.TOTL','SL.AGR.EMPL.ZS','NY.GDP.MKTP.CD',
            'DPANUSSPF','EG.ELC.ACCS.UR.ZS','EG.ELC.ACCS.RU.ZS']
countryAll = ';'.join(['BR','RU','IN','CN','ZA'])

df_list = []
for indicator_type in indicator_list:
    text = 'http://api.worldbank.org/v2/country/{}/indicator/{}?format=json&per_page=1000&date=2012:2017'.format(countryAll,
                                                                                indicator_type)
    temp = getApiData(text)[1]
    df = pd.read_json(json.dumps(temp),orient='columns')
    df = df.drop(['unit','obs_status','decimal','countryiso3code'],axis=1)
    df['country'] = df['country'].apply(lambda x:x['value'])
    df['indicator'] = df['indicator'].apply(lambda x:x['value'])
    df = df.rename(columns={'value':df.indicator[0]})
    df = df.drop(['indicator'],axis=1)
    df_list.append(df)
dfAns = df_list[0]
for df in df_list:
    dfAns = pd.merge(dfAns,df,on=['country','date'],how='outer')
dfAns = dfAns.T.drop_duplicates().T
dfAns = dfAns.rename(columns={'Population, total_x':'Population, total'})
dfAns.head()

,country,date,"Population, total",Employment in agriculture (% of total employment) (modeled ILO estimate),GDP (current US$),"Exchange rate, old LCU per USD extended forward, period average,,","Access to electricity, urban (% of urban population)","Access to electricity, rural (% of rural population)"
0,Brazil,2017,207833831,9.502,2.05359e+12,3.19026,100,100
1,Brazil,2016,206163058,10.182,1.79628e+12,3.48671,99.9,98.4671
2,Brazil,2015,204471769,10.242,1.80221e+12,3.33169,99.9562,98.2324
3,Brazil,2014,202763735,10.371,2.45599e+12,2.35427,99.9456,97.9101
4,Brazil,2013,201035903,11.199,2.47281e+12,2.15873,99.935,97.502


In [108]:
graph_predict_temperature = []
graph_predict_temperature.append(
    go.Scatter(x = df.time,
               y = df.temperatureC,
               mode = 'lines'
        )
)
layout_predict_temperature = dict(title='Temperature In 48 Hours', 
                                  xaxis=dict(title='time(h)'),
                                  yaxis=dict(title='degrees'))
fig = go.Figure(go.Scatter(
        x = df.time,
        y = df.temperatureC,
        mode = 'lines'
    ),layout_predict_temperature)
fig.show()

AttributeError: 'DataFrame' object has no attribute 'time'

In [ ]:

go.Scatter()

Building jupyterlab assets (build:prod:minimize)
-

In [ ]:
conver